In [1]:
!pip install tensorflow==2.5.1 tensorflow-gpu==2.5.1 opencv-python mediapipe sklearn matplotlib

You should consider upgrading via the 'c:\users\hp\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [14]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        draw_styled_landmarks(image, results)
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [7]:
cap.release()
cv2.destroyAllWindows()

In [8]:
def extract_keypoints(results):
    if(results.left_hand_landmarks):
        pose = []
        for res in results.pose_landmarks.landmark:
            test = np.array([res.x, res.y, res.z, res.visibility])
            pose.append(test)
        pose=np.array(pose)
        pose=pose.flatten()
    else :
        pose=np.zeros(132)
    if(results.left_hand_landmarks) :
        lh=[]
        for res in results.left_hand_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            lh.append(test)
        lh=np.array(lh)
        lh=lh.flatten()
    else :
        lh=np.zeros(21*3)
    if(results.right_hand_landmarks) :
        rh=[]
        for res in results.right_hand_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            rh.append(test)
        rh=np.array(rh)
        rh=rh.flatten()
    else :
        rh=np.zeros(21*3)   
    if(results.face_landmarks) :
        face=[]
        for res in results.face_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            face.append(test)
        face=np.array(face)
        face=face.flatten()
    else :
        face=np.zeros(468*3)   
    return np.concatenate([pose, face, lh, rh])

In [9]:
len(extract_keypoints(results))

1662

In [12]:
DATA_PATH = os.path.join('MP_Data') 
actions = np.array(['hello', 'thanks', 'iloveyou'])
no_sequences = 30
sequence_length = 30
start_folder = 30

In [13]:
for action in actions: 
    #dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [15]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [16]:
cap.release()
cv2.destroyAllWindows()

In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [18]:
label_map = {label:num for num, label in enumerate(actions)}

In [19]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [20]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [21]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [23]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [24]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=974, callbacks=[tb_callback])

Epoch 1/974
3/3 [==============================] - 6s 344ms/step - loss: 1.7724 - categorical_accuracy: 0.2235
Epoch 2/974
3/3 [==============================] - 0s 90ms/step - loss: 8.0550 - categorical_accuracy: 0.4000
Epoch 3/974
3/3 [==============================] - 0s 83ms/step - loss: 19.4546 - categorical_accuracy: 0.3529
Epoch 4/974
3/3 [==============================] - 0s 83ms/step - loss: 35.5148 - categorical_accuracy: 0.3529
Epoch 5/974
3/3 [==============================] - 0s 84ms/step - loss: 56.6157 - categorical_accuracy: 0.2824
Epoch 6/974
3/3 [==============================] - 0s 82ms/step - loss: 24.0613 - categorical_accuracy: 0.3882
Epoch 7/974
3/3 [==============================] - 0s 81ms/step - loss: 21.9192 - categorical_accuracy: 0.2706
Epoch 8/974
3/3 [==============================] - 0s 83ms/step - loss: 17.0031 - categorical_accuracy: 0.3059
Epoch 9/974
3/3 [==============================] - 0s 86ms/step - loss: 22.3209 - categorical_accuracy: 0.2824
Ep

3/3 [==============================] - 0s 85ms/step - loss: 0.7028 - categorical_accuracy: 0.6353
Epoch 75/974
3/3 [==============================] - 0s 108ms/step - loss: 0.6933 - categorical_accuracy: 0.6353
Epoch 76/974
3/3 [==============================] - 0s 109ms/step - loss: 0.6626 - categorical_accuracy: 0.7294
Epoch 77/974
3/3 [==============================] - 0s 91ms/step - loss: 0.6446 - categorical_accuracy: 0.7059
Epoch 78/974
3/3 [==============================] - 0s 102ms/step - loss: 0.6204 - categorical_accuracy: 0.7529
Epoch 79/974
3/3 [==============================] - 0s 104ms/step - loss: 0.6050 - categorical_accuracy: 0.7647
Epoch 80/974
3/3 [==============================] - 0s 91ms/step - loss: 0.5869 - categorical_accuracy: 0.7765
Epoch 81/974
3/3 [==============================] - 0s 115ms/step - loss: 0.5686 - categorical_accuracy: 0.7882
Epoch 82/974
3/3 [==============================] - 0s 104ms/step - loss: 0.5564 - categorical_accuracy: 0.7882
Epoch 83

3/3 [==============================] - 0s 107ms/step - loss: 0.3392 - categorical_accuracy: 0.8471
Epoch 148/974
3/3 [==============================] - 0s 104ms/step - loss: 0.3374 - categorical_accuracy: 0.8471
Epoch 149/974
3/3 [==============================] - 0s 100ms/step - loss: 0.3346 - categorical_accuracy: 0.8471
Epoch 150/974
3/3 [==============================] - 0s 109ms/step - loss: 0.3350 - categorical_accuracy: 0.8588
Epoch 151/974
3/3 [==============================] - 0s 78ms/step - loss: 0.3314 - categorical_accuracy: 0.8588
Epoch 152/974
3/3 [==============================] - 0s 108ms/step - loss: 0.3281 - categorical_accuracy: 0.8588
Epoch 153/974
3/3 [==============================] - 0s 87ms/step - loss: 0.3306 - categorical_accuracy: 0.8706
Epoch 154/974
3/3 [==============================] - 0s 104ms/step - loss: 0.3246 - categorical_accuracy: 0.8588
Epoch 155/974
3/3 [==============================] - 0s 105ms/step - loss: 0.3220 - categorical_accuracy: 0.8588

3/3 [==============================] - 0s 80ms/step - loss: 0.3835 - categorical_accuracy: 0.8588
Epoch 220/974
3/3 [==============================] - 0s 82ms/step - loss: 0.3784 - categorical_accuracy: 0.8588
Epoch 221/974
3/3 [==============================] - 0s 85ms/step - loss: 0.3768 - categorical_accuracy: 0.8706
Epoch 222/974
3/3 [==============================] - 0s 94ms/step - loss: 0.3744 - categorical_accuracy: 0.8588
Epoch 223/974
3/3 [==============================] - 0s 96ms/step - loss: 0.3724 - categorical_accuracy: 0.8588
Epoch 224/974
3/3 [==============================] - 0s 125ms/step - loss: 0.3666 - categorical_accuracy: 0.8588
Epoch 225/974
3/3 [==============================] - 0s 98ms/step - loss: 0.3600 - categorical_accuracy: 0.8706
Epoch 226/974
3/3 [==============================] - 0s 97ms/step - loss: 0.3576 - categorical_accuracy: 0.8588
Epoch 227/974
3/3 [==============================] - 0s 110ms/step - loss: 0.3567 - categorical_accuracy: 0.8706
Epoc

Epoch 292/974
3/3 [==============================] - 0s 86ms/step - loss: 0.3112 - categorical_accuracy: 0.8471
Epoch 293/974
3/3 [==============================] - 0s 102ms/step - loss: 0.2939 - categorical_accuracy: 0.8824
Epoch 294/974
3/3 [==============================] - 0s 104ms/step - loss: 0.2580 - categorical_accuracy: 0.8824
Epoch 295/974
3/3 [==============================] - 0s 89ms/step - loss: 0.2426 - categorical_accuracy: 0.9059
Epoch 296/974
3/3 [==============================] - 0s 114ms/step - loss: 0.2337 - categorical_accuracy: 0.9529
Epoch 297/974
3/3 [==============================] - 0s 84ms/step - loss: 0.2245 - categorical_accuracy: 0.8941
Epoch 298/974
3/3 [==============================] - 0s 90ms/step - loss: 0.2262 - categorical_accuracy: 0.9176
Epoch 299/974
3/3 [==============================] - 0s 97ms/step - loss: 0.2099 - categorical_accuracy: 0.9529
Epoch 300/974
3/3 [==============================] - 0s 87ms/step - loss: 0.2193 - categorical_accura

...

Epoch 655/974
3/3 [==============================] - 0s 98ms/step - loss: 0.1521 - categorical_accuracy: 0.9529
Epoch 656/974
3/3 [==============================] - 0s 85ms/step - loss: 0.1407 - categorical_accuracy: 0.9647
Epoch 657/974
3/3 [==============================] - 0s 110ms/step - loss: 0.1758 - categorical_accuracy: 0.9059
Epoch 658/974
3/3 [==============================] - 0s 103ms/step - loss: 0.1317 - categorical_accuracy: 0.9529
Epoch 659/974
3/3 [==============================] - 0s 85ms/step - loss: 0.1953 - categorical_accuracy: 0.9412
Epoch 660/974
3/3 [==============================] - 0s 99ms/step - loss: 0.2027 - categorical_accuracy: 0.9412
Epoch 661/974
3/3 [==============================] - 0s 90ms/step - loss: 0.2660 - categorical_accuracy: 0.8824
Epoch 662/974
3/3 [==============================] - 0s 97ms/step - loss: 0.2786 - categorical_accuracy: 0.8941
Epoch 663/974
3/3 [==============================] - 0s 78ms/step - loss: 0.2624 - categorical_accurac

Epoch 728/974
3/3 [==============================] - 0s 101ms/step - loss: 0.1033 - categorical_accuracy: 0.9765
Epoch 729/974
3/3 [==============================] - 0s 104ms/step - loss: 0.1430 - categorical_accuracy: 0.9176
Epoch 730/974
3/3 [==============================] - 0s 94ms/step - loss: 0.1642 - categorical_accuracy: 0.9176
Epoch 731/974
3/3 [==============================] - 0s 96ms/step - loss: 0.2161 - categorical_accuracy: 0.8706
Epoch 732/974
3/3 [==============================] - 0s 107ms/step - loss: 0.1484 - categorical_accuracy: 0.9412
Epoch 733/974
3/3 [==============================] - 0s 85ms/step - loss: 0.0967 - categorical_accuracy: 0.9529
Epoch 734/974
3/3 [==============================] - 0s 93ms/step - loss: 0.1100 - categorical_accuracy: 0.9647
Epoch 735/974
3/3 [==============================] - 0s 103ms/step - loss: 0.1160 - categorical_accuracy: 0.9294
Epoch 736/974
3/3 [==============================] - 0s 92ms/step - loss: 0.0746 - categorical_accur

Epoch 801/974
3/3 [==============================] - 0s 108ms/step - loss: 0.0238 - categorical_accuracy: 0.9882
Epoch 802/974
3/3 [==============================] - 0s 109ms/step - loss: 0.0206 - categorical_accuracy: 1.0000
Epoch 803/974
3/3 [==============================] - 0s 88ms/step - loss: 0.0197 - categorical_accuracy: 1.0000
Epoch 804/974
3/3 [==============================] - 0s 106ms/step - loss: 0.0182 - categorical_accuracy: 1.0000
Epoch 805/974
3/3 [==============================] - 0s 80ms/step - loss: 0.0195 - categorical_accuracy: 1.0000
Epoch 806/974
3/3 [==============================] - 0s 93ms/step - loss: 0.0322 - categorical_accuracy: 0.9882
Epoch 807/974
3/3 [==============================] - 0s 88ms/step - loss: 0.0611 - categorical_accuracy: 0.9882
Epoch 808/974
3/3 [==============================] - 0s 102ms/step - loss: 0.0418 - categorical_accuracy: 0.9765
Epoch 809/974
3/3 [==============================] - 0s 88ms/step - loss: 0.0327 - categorical_accur

3/3 [==============================] - 0s 93ms/step - loss: 0.0120 - categorical_accuracy: 1.0000
Epoch 874/974
3/3 [==============================] - 0s 91ms/step - loss: 0.0144 - categorical_accuracy: 1.0000
Epoch 875/974
3/3 [==============================] - 0s 111ms/step - loss: 0.0130 - categorical_accuracy: 1.0000
Epoch 876/974
3/3 [==============================] - 0s 89ms/step - loss: 0.0144 - categorical_accuracy: 1.0000
Epoch 877/974
3/3 [==============================] - 0s 93ms/step - loss: 0.0115 - categorical_accuracy: 1.0000
Epoch 878/974
3/3 [==============================] - 0s 95ms/step - loss: 0.0118 - categorical_accuracy: 1.0000
Epoch 879/974
3/3 [==============================] - 0s 101ms/step - loss: 0.0154 - categorical_accuracy: 1.0000
Epoch 880/974
3/3 [==============================] - 0s 88ms/step - loss: 0.0087 - categorical_accuracy: 1.0000
Epoch 881/974
3/3 [==============================] - 0s 93ms/step - loss: 0.0127 - categorical_accuracy: 1.0000
Epoc

3/3 [==============================] - 0s 98ms/step - loss: 0.0035 - categorical_accuracy: 1.0000
Epoch 947/974
3/3 [==============================] - 0s 91ms/step - loss: 0.0035 - categorical_accuracy: 1.0000
Epoch 948/974
3/3 [==============================] - 0s 95ms/step - loss: 0.0031 - categorical_accuracy: 1.0000
Epoch 949/974
3/3 [==============================] - 0s 94ms/step - loss: 0.0033 - categorical_accuracy: 1.0000
Epoch 950/974
3/3 [==============================] - 0s 96ms/step - loss: 0.0033 - categorical_accuracy: 1.0000
Epoch 951/974
3/3 [==============================] - 0s 96ms/step - loss: 0.0031 - categorical_accuracy: 1.0000
Epoch 952/974
3/3 [==============================] - 0s 91ms/step - loss: 0.0035 - categorical_accuracy: 1.0000
Epoch 953/974
3/3 [==============================] - 0s 113ms/step - loss: 0.0032 - categorical_accuracy: 1.0000
Epoch 954/974
3/3 [==============================] - 0s 90ms/step - loss: 0.0030 - categorical_accuracy: 1.0000
Epoch

In [27]:
model.save('action.h5')

In [28]:
model.load_weights('action.h5')

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

In [30]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 0],
        [3, 0]],

       [[2, 2],
        [0, 1]],

       [[3, 1],
        [0, 1]]], dtype=int64)

In [31]:
accuracy_score(ytrue, yhat)

0.4

In [32]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [35]:
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        draw_styled_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'medi

In [34]:
cap.release()
cv2.destroyAllWindows()